# Semantic Chunking Evaluation — RAG Pipeline

This notebook evaluates a RAG system using **semantic chunking** instead of standard length-based chunking.
Semantic chunking splits the document into meaning-preserving units based on the actual content, which helps improve retrieval relevance and overall answer grounding.

Only the chunking method differs from other implementations — all other components in the pipeline remain consistent.


In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Basic_RAG_Semantic_Chunking"


# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

Environment Variables Set:
LANGSMITH_TRACING: true
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: Basic_RAG_Semantic_Chunking


In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


In [ ]:
import langchain
from langchain_community.document_loaders import PyPDFLoader
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

## LLM and Embedding Model

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=4000,
)

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

## Loading the PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

def load_pdf(file_path: str):
    loader = PyPDFLoader(file_path)
    return loader.load()

# Load the PDF document
path = 'NVIDIA_Report_2024.pdf'
documents = load_pdf(path)

# Create a Semantic Chunker
text_splitter = SemanticChunker(embeddings)

# Extract text content from documents
text_contents = [doc.page_content for doc in documents]

# Chunk the document based on semantic similarity
chunked_documents = text_splitter.create_documents(text_contents)

# Print the chunked content
# for i, chunk in enumerate(chunked_documents):
#     print(f"Chunk {i + 1}:")
#     print(chunk)
#     print("====")

In [8]:
docs = chunked_documents

## Creating FAISS, local persistence

In [9]:
def create_vector_store(docs):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local("nvidia_chunk_semantic")
    return vectorstore.as_retriever()


create_vector_store(docs)

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11f986c50>, search_kwargs={})

## Reloading saved FAISS

In [10]:
# Load the index back

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore_semantic = FAISS.load_local(
    "nvidia_chunk_semantic", 
    embeddings,  # Use the same embeddings used during creation
    allow_dangerous_deserialization=True  # Important for loading
)


retriever_semantic = vectorstore_semantic.as_retriever()

## Basic RAG pipeline 

In [11]:
from langsmith import traceable

def setup_rag_pipeline():
    template = """"
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # Setup RAG pipeline
    return (
        {"context": retriever_semantic, "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# Initialize the RAG chain with tracing
rag_chain = setup_rag_pipeline()


## RAG Query, Retrieved Docs and Answer 

In [12]:
import pandas as pd
from IPython.display import display

# Define query_rag function with retrieval and display logic
@traceable
def query_rag(question: str, top_k=4):
    """
    Query the RAG pipeline, retrieve relevant documents, and display their similarity scores in descending order.
    Returns both the generated answer and retrieved contexts.
    """
    # Retrieve the top-k chunks with similarity scores
    retrieved_docs_with_scores = retriever_semantic.vectorstore.similarity_search_with_score(question, k=top_k)

    # Sort documents by similarity score in descending order
    retrieved_docs_with_scores = sorted(retrieved_docs_with_scores, key=lambda x: x[1], reverse=True)

    # Extract document details into a structured DataFrame
    df = pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),  # Rounding for readability
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters for clarity
        }
        for doc, score in retrieved_docs_with_scores
    ])

    # Ensure full content is displayed in the DataFrame
    pd.set_option("display.max_colwidth", None)  # Do not truncate column content
    pd.set_option("display.width", 1000)  # Set a larger display width for better visibility

    # Display the DataFrame in Jupyter
    display(df)

    # Extract contexts (raw document texts)
    retrieved_contexts = [doc.page_content for doc, _ in retrieved_docs_with_scores]

    # Run the RAG pipeline with the retrieved context
    answer = rag_chain.invoke(question)

    # Return both the answer and the retrieved contexts
    return {
        "answer": answer,
        "contexts": retrieved_contexts
    }

/Users/user/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [13]:
# Example usage:
response = query_rag("Who is the CEO of the firm?")
print(response["answer"])  # Print the answer
print("\nContexts:", response["contexts"])  # Print the retrieved contexts

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,1.2736,"Timothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General Counsel andSecretary in February 2018. Prior to NVIDIA, Mr. Teter spent more than two decades at the law firm of Cooley LLP, where he focused on litigating patent and\ntechnology related matters. Prior to attending law school, he worked as an engineer at Lockheed Missiles and Space Company, an aerospace company. Mr.Teter holds a B.S. degree in Mechanical Engineering from the University of California at Davis and a J.D...."
1,Unknown Document,N/A,1.2154,Kress\nExecutive Vice President and Chief Financial Officer...
2,Unknown Document,N/A,1.2056,"Jones \n/s/ MELISSA B. LORA Director February 21, 2024\nMelissa B. Lora\n/s/ MICHAEL MCCAFFERY Director February 21, 2024\nMichael McCaffery\n/s/ STEPHEN C. NEAL Director February 21, 2024\nStephen C. Neal\n/s/ MARK L. PERRY Director February 21, 2024\nMark L. Perry \n/s/ A. BROOKE SEAWELL Director February 21, 2024\nA. Brooke Seawell \n/s/ AARTI SHAH Director February 21, 2024\nAarti Shah\n/s/ MARK STEVENS Director February 21, 2024\nMark Stevens \n85..."
3,Unknown Document,N/A,1.1613,"Table of Contents\nOur support is enhanced during times of crisis, such as war or economic volatility, to take care of our existing team of world-class talent and their families. Diversity, Inclusion, and Belonging\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees. When recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in thetechnology field, including women, Black/African American, and Hispanic/Latino candidates. To this end, we have been:\n• Partnering with institutions and professional organizations serving historically underrepresented communities;\n• Embedding dedicated recruiting teams to business areas to shepherd underrepresented candidates through the interview process and find internalopportunities;\n• Supporting the development of women employees through programs aimed at building a pipeline of future leader..."


The CEO of the firm is Jen-Hsun Huang.

Contexts: ['Timothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General Counsel andSecretary in February 2018. Prior to NVIDIA, Mr. Teter spent more than two decades at the law firm of Cooley LLP, where he focused on litigating patent and\ntechnology related matters. Prior to attending law school, he worked as an engineer at Lockheed Missiles and Space Company, an aerospace company. Mr.Teter holds a B.S. degree in Mechanical Engineering from the University of California at Davis and a J.D.', 'Kress\nExecutive Vice President and Chief Financial Officer', 'Jones  \n/s/ MELISSA B. LORA Director February 21, 2024\nMelissa B. Lora\n/s/ MICHAEL MCCAFFERY Director February 21, 2024\nMichael McCaffery\n/s/ STEPHEN C. NEAL Director February 21, 2024\nStephen C. Neal\n/s/ MARK L. PERRY Director February 21, 2024\nMark L. Perry  \n/s/ A. BROOKE SEAWELL Director February 21, 2024

## UI with Gradio

In [14]:
import gradio as gr

In [15]:
def chat_with_rag(user_input):
    response = query_rag(user_input)  # Call your existing RAG pipeline
    return response["answer"]  # Return the generated answer

In [16]:
# Create a simple Gradio chat interface
with gr.Blocks() as demo:
    gr.Markdown("# Chat with NVIDIA RAG AI")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Kérdezz bármit:")
    
    # Button to submit query
    btn = gr.Button("Kérdezz")

    # Append user message and system response
    def respond(history, message):
        response = chat_with_rag(message)
        history.append((message, response))  # Append conversation to chat
        return history, ""

    btn.click(respond, [chatbot, msg], [chatbot, msg])

# Launch the Gradio app
demo.launch(share=True)


/Users/user/anaconda3/lib/python3.11/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://93cb6c3c41b3643814.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,1.2801,"Timothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General Counsel andSecretary in February 2018. Prior to NVIDIA, Mr. Teter spent more than two decades at the law firm of Cooley LLP, where he focused on litigating patent and\ntechnology related matters. Prior to attending law school, he worked as an engineer at Lockheed Missiles and Space Company, an aerospace company. Mr.Teter holds a B.S. degree in Mechanical Engineering from the University of California at Davis and a J.D...."
1,Unknown Document,N/A,1.2646,Kress\nExecutive Vice President and Chief Financial Officer...
2,Unknown Document,N/A,1.1821,"Jones \n/s/ MELISSA B. LORA Director February 21, 2024\nMelissa B. Lora\n/s/ MICHAEL MCCAFFERY Director February 21, 2024\nMichael McCaffery\n/s/ STEPHEN C. NEAL Director February 21, 2024\nStephen C. Neal\n/s/ MARK L. PERRY Director February 21, 2024\nMark L. Perry \n/s/ A. BROOKE SEAWELL Director February 21, 2024\nA. Brooke Seawell \n/s/ AARTI SHAH Director February 21, 2024\nAarti Shah\n/s/ MARK STEVENS Director February 21, 2024\nMark Stevens \n85..."
3,Unknown Document,N/A,1.1160,"Table of Contents\nOur support is enhanced during times of crisis, such as war or economic volatility, to take care of our existing team of world-class talent and their families. Diversity, Inclusion, and Belonging\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees. When recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in thetechnology field, including women, Black/African American, and Hispanic/Latino candidates. To this end, we have been:\n• Partnering with institutions and professional organizations serving historically underrepresented communities;\n• Embedding dedicated recruiting teams to business areas to shepherd underrepresented candidates through the interview process and find internalopportunities;\n• Supporting the development of women employees through programs aimed at building a pipeline of future leader..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.8299,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive. Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago. Data Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including aut..."
1,Unknown Document,N/A,0.8156,"Table of Contents\nNVIDIA Corporation and SubsidiariesNotes to the Consolidated Financial Statements\n(Continued)\nThe tax effect of temporary differences that gives rise to significant portions of the deferred tax assets and liabilities are presented below:\n Jan 28, 2024 Jan 29, 2023\n (In millions)\nDeferred tax assets: \nCapitalized research and development expenditure $ 3,376 $ 1,859 \nGILTI deferred tax assets 1,576 800 \nAccruals and reserves, not currently deductible for tax purposes 1,121 686 \nResearch and other tax credit carryforwards 936 951 \nNet operating loss and capital loss carryforwards 439 409 \nOperating lease liabilities 263 193 \nStock-based compensation 106 99 \nProperty, equipment and intangible assets 4 66 \nOther deferred tax assets 179 91 \nGross deferred tax assets 8,000 5,154 \nLess valuation allowance (1,552) (1,484)\nTotal deferred tax assets 6,448 3,670 \nDeferred tax liabilities: \nUnremitted earnings of foreign subsidiaries (502) (228)\nOperating lease assets (255) (17..."
2,Unknown Document,N/A,0.7560,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements(Continued)\nNote 14 - Income Taxes\nThe income tax expense (benefit) applicable to income before income taxes consists of the following:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\n (In millions)\nCurrent income taxes: \nFederal $ 5,710 $ 1,703 $ 482 \nState 335 46 42 \nForeign 502 228 71 \nTotal current 6,547 1,977 595 \nDeferred income taxes: \nFederal (2,499) (2,165) (420)\nState (206) — — \nForeign 216 1 14 \nTotal deferred (2,489) (2,164) (406)\nIncome tax expense (benefit) $ 4,058 $ (187)$ 189 \nIncome before income tax consists of the following:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\n (In millions)\nU.S. $ 29,495 $ 3,477 $ 8,446 \nForeign 4,323 704 1,495 \nIncome before income tax $ 33,818 $ 4,181 $ 9,941 \nThe income tax expense (benefit) differs from the amount computed by applying the U.S...."
3,Unknown Document,N/A,0.7552,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752 \nNet income per share:\nBasic

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.8157,"Source: FactSet financial data and analytics. 1/27/2019 1/26/2020 1/31/2021 1/30/2022 1/29/2023 1/28/2024\nNVIDIA Corporation $ 100.00 $ 157.02 $ 326.26 $ 574.15 $ 512.40 $ 1,536.28 \nS&P 500 $ 100.00 $ 126.17 $ 144.83 $ 175.25 $ 163.63 $ 199.83 \nNasdaq 100 $ 100.00 $ 136.15 $ 194.20 $ 218.68 $ 185.67 $ 268.13 \nItem 6. [Reserved]\n33..."
1,Unknown Document,N/A,0.7839,"Table of Contents\nNVIDIA Corporation and SubsidiariesNotes to the Consolidated Financial Statements(Continued)\n Jan 28, 2024 Jan 29, 2023 EstimatedUseful Life\n(In millions) (In years)\nProperty and Equipment:\nLand $ 218 $ 218 (A)\nBuildings, leasehold improvements, and furniture 1,816 1,598 (B)\nEquipment, compute hardware, and software 5,200 4,303 3-7\nConstruction in process 189 382 (C)\nTotal property and equipment, gross 7,423 6,501 \nAccumulated depreciation and amortization (3,509) (2,694) \nTotal property and equipment, net $ 3,914 $ 3,807 \n(A) Land is a non-depreciable asset. (B) The estimated useful lives of our buildings are up to thirty years...."
2,Unknown Document,N/A,0.7792,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements(Continued)\nThe following table presents summarized information for long-lived assets by country. Long-lived assets consist of property and equipment and exclude otherassets, operating lease assets, goodwill, and intangible assets...."
3,Unknown Document,N/A,0.7091,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Balance Sheets(In millions, except par value)\n Jan 28, 2024 Jan 29, 2023\nAssets \nCurrent assets: \nCash and cash equivalents $ 7,280 $ 3,389 \nMarketable securities 18,704 9,907 \nAccounts receivable, net 9,999 3,827 \nInventories 5,282 5,159 \nPrepaid expenses and other current assets 3,080 791 \nTotal current assets 44,345 23,073 \nProperty and equipment, net 3,914 3,807 \nOperating lease assets 1,346 1,038 \nGoodwill 4,430 4,372 \nIntangible assets, net 1,112 1,676 \nDeferred income tax assets 6,081 3,396 \nOther assets 4,500 3,820 \nTotal assets $ 65,728 $ 41,182 \nLiabilities and Shareholders' Equity\nCurrent liabilities: \nAccounts payable $ 2,699 $ 1,193 \nAccrued and other current liabilities 6,682 4,120 \nShort-term debt 1,250 1,250 \nTotal current liabilities 10,631 6,563 \nLong-term debt 8,459 9,703 \nLong-term operating lease liabilities 1,119 902 \nOther long-term liabilities 2,541 1,913 \nTotal liabilities 22,750 19,081 \nCo..."


## RAG Evaluators

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [17]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [27]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_8984/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [28]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"])
    return {"answer": response["answer"]}

In [29]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [30]:
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [31]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
#     llm = ChatOpenAI(model="gpt-4o", temperature=0)
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [30]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="Basic_RAG_1400",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Basic_RAG_1400-397337c3' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=e00ec39c-16db-40be-81a1-09489f9761e3




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,12,0.8107,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr.\nPuri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinne..."
1,NVIDIA_Report_2024.pdf,12,0.7975,"Debora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement\nsolutions and optical products for the telecommunications industry. She served from 2002 to 2004 as Senior Vice President and General Manager of the Electro-Optics business at Coherent, Inc., a manufacturer of commercial and scientific laser equipment. Previously, she worked at Quantum Corp., a data protectioncompany, as President of the Personal Computer Hard Disk Drive Division, and at Hewlett-Packard. Ms. Shoquist holds a B.S. degree in Electrical Engineering\nfrom Kansas State University and a B.S. degree in Biology from Santa Clara University.\nTimothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General C..."
2,NVIDIA_Report_2024.pdf,11,0.7890,"• Providing training and education to managers and peers on fostering supportive environments and recruiting for diversity;\n• Track equity and parity in retention, promotions, pay, and employee engagement scores; and\n• Measuring year over year progress and providing leadership visibility on diversity efforts.\nAs of the end of fiscal year 2024, our global workforce was 79% male, 20% female, and 1% not declared, with 6% of our workforce in the United States\ncomposed of Black or African American and Hispanic or Latino employees.\nFlexible Working Environment\nWe support a flexible work environment, understanding that many employees want the ability to work from home under certain conditions. This flexibility\nsupports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformat..."
3,NVIDIA_Report_2024.pdf,11,0.5696,"Name Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocesso

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,0.6756,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752 \nNet income per share:\nBasic $ 12.05 $ 1.76 $ 3.91 \nDiluted $ 11.93 $ 1.74 $ 3.85 \nWeighted average shares used in per share computation:\nBasic 2,469 2,487 2,496 \nDiluted 2,494 2,507 2,535 \nSee accompanying notes to ..."
1,NVIDIA_Report_2024.pdf,35,0.6607,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,0.6226,"Year Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 32,016 $ 5,083 $ 26,933 530 %\nGraphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."
3,NVIDIA_Report_2024.pdf,35,0.5638,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nRevenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including aut..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,5,0.6688,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators.\nOur gaming platforms leverage our GPUs and sophisticated software to enhance the gaming experience with smoother, higher quality graphics. We developed\nNVIDIA RTX to bring next generation graphics and AI to games. NVIDIA RTX features ray tracing technology for real-time, cinematic-quality rendering. Raytracing, which ha..."
1,NVIDIA_Report_2024.pdf,4,0.6481,"broader set of PC users to choose NVIDIA GPUs for running generative AI applications locally on their PC, which is critical for privacy, latency, and cost-sensitiveAI applications.\nProfessional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; o..."
2,NVIDIA_Report_2024.pdf,3,0.5683,"NVIDIA has a platform strategy, bringing together hardware, systems, software, algorithms, libraries, and services to create unique value for the markets we\nserve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern\ncomputing. Our invention of the GPU in 1999 sparked the growth of the PC gaming m..."
3,NVIDIA_Report_2024.pdf,4,0.5541,"embedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for\ninfotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platfor

### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [32]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [33]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
#     llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

In [33]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="Basic_RAG_1400",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Basic_RAG_1400-876d5c87' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=5e81f4ea-1429-460f-902c-ccfe5f95b25d




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,12,0.8108,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr.\nPuri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinne..."
1,NVIDIA_Report_2024.pdf,12,0.7976,"Debora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement\nsolutions and optical products for the telecommunications industry. She served from 2002 to 2004 as Senior Vice President and General Manager of the Electro-Optics business at Coherent, Inc., a manufacturer of commercial and scientific laser equipment. Previously, she worked at Quantum Corp., a data protectioncompany, as President of the Personal Computer Hard Disk Drive Division, and at Hewlett-Packard. Ms. Shoquist holds a B.S. degree in Electrical Engineering\nfrom Kansas State University and a B.S. degree in Biology from Santa Clara University.\nTimothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General C..."
2,NVIDIA_Report_2024.pdf,11,0.7891,"• Providing training and education to managers and peers on fostering supportive environments and recruiting for diversity;\n• Track equity and parity in retention, promotions, pay, and employee engagement scores; and\n• Measuring year over year progress and providing leadership visibility on diversity efforts.\nAs of the end of fiscal year 2024, our global workforce was 79% male, 20% female, and 1% not declared, with 6% of our workforce in the United States\ncomposed of Black or African American and Hispanic or Latino employees.\nFlexible Working Environment\nWe support a flexible work environment, understanding that many employees want the ability to work from home under certain conditions. This flexibility\nsupports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformat..."
3,NVIDIA_Report_2024.pdf,11,0.5697,"Name Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocesso

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,0.6755,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752 \nNet income per share:\nBasic $ 12.05 $ 1.76 $ 3.91 \nDiluted $ 11.93 $ 1.74 $ 3.85 \nWeighted average shares used in per share computation:\nBasic 2,469 2,487 2,496 \nDiluted 2,494 2,507 2,535 \nSee accompanying notes to ..."
1,NVIDIA_Report_2024.pdf,35,0.6607,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,0.6225,"Year Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 32,016 $ 5,083 $ 26,933 530 %\nGraphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."
3,NVIDIA_Report_2024.pdf,35,0.5638,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nRevenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including aut..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,5,0.6688,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators.\nOur gaming platforms leverage our GPUs and sophisticated software to enhance the gaming experience with smoother, higher quality graphics. We developed\nNVIDIA RTX to bring next generation graphics and AI to games. NVIDIA RTX features ray tracing technology for real-time, cinematic-quality rendering. Raytracing, which ha..."
1,NVIDIA_Report_2024.pdf,4,0.6481,"broader set of PC users to choose NVIDIA GPUs for running generative AI applications locally on their PC, which is critical for privacy, latency, and cost-sensitiveAI applications.\nProfessional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; o..."
2,NVIDIA_Report_2024.pdf,3,0.5683,"NVIDIA has a platform strategy, bringing together hardware, systems, software, algorithms, libraries, and services to create unique value for the markets we\nserve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern\ncomputing. Our invention of the GPU in 1999 sparked the growth of the PC gaming m..."
3,NVIDIA_Report_2024.pdf,4,0.5541,"embedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for\ninfotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platfor

### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [34]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [35]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
#     llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [36]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="Basic_RAG_1400",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Basic_RAG_500-3b94c7a8' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=4386f479-819b-4b08-8408-8d12d831532d




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,5,0.6688,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators.\nOur gaming platforms leverage our GPUs and sophisticated software to enhance the gaming experience with smoother, higher quality graphics. We developed\nNVIDIA RTX to bring next generation graphics and AI to games. NVIDIA RTX features ray tracing technology for real-time, cinematic-quality rendering. Raytracing, which ha..."
1,NVIDIA_Report_2024.pdf,4,0.6481,"broader set of PC users to choose NVIDIA GPUs for running generative AI applications locally on their PC, which is critical for privacy, latency, and cost-sensitiveAI applications.\nProfessional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; o..."
2,NVIDIA_Report_2024.pdf,3,0.5683,"NVIDIA has a platform strategy, bringing together hardware, systems, software, algorithms, libraries, and services to create unique value for the markets we\nserve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern\ncomputing. Our invention of the GPU in 1999 sparked the growth of the PC gaming m..."
3,NVIDIA_Report_2024.pdf,4,0.5541,"embedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for\ninfotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platfor

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,12,0.8115,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr.\nPuri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinne..."
1,NVIDIA_Report_2024.pdf,12,0.7973,"Debora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement\nsolutions and optical products for the telecommunications industry. She served from 2002 to 2004 as Senior Vice President and General Manager of the Electro-Optics business at Coherent, Inc., a manufacturer of commercial and scientific laser equipment. Previously, she worked at Quantum Corp., a data protectioncompany, as President of the Personal Computer Hard Disk Drive Division, and at Hewlett-Packard. Ms. Shoquist holds a B.S. degree in Electrical Engineering\nfrom Kansas State University and a B.S. degree in Biology from Santa Clara University.\nTimothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General C..."
2,NVIDIA_Report_2024.pdf,11,0.7887,"• Providing training and education to managers and peers on fostering supportive environments and recruiting for diversity;\n• Track equity and parity in retention, promotions, pay, and employee engagement scores; and\n• Measuring year over year progress and providing leadership visibility on diversity efforts.\nAs of the end of fiscal year 2024, our global workforce was 79% male, 20% female, and 1% not declared, with 6% of our workforce in the United States\ncomposed of Black or African American and Hispanic or Latino employees.\nFlexible Working Environment\nWe support a flexible work environment, understanding that many employees want the ability to work from home under certain conditions. This flexibility\nsupports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformat..."
3,NVIDIA_Report_2024.pdf,11,0.5697,"Name Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocesso

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,0.6756,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752 \nNet income per share:\nBasic $ 12.05 $ 1.76 $ 3.91 \nDiluted $ 11.93 $ 1.74 $ 3.85 \nWeighted average shares used in per share computation:\nBasic 2,469 2,487 2,496 \nDiluted 2,494 2,507 2,535 \nSee accompanying notes to ..."
1,NVIDIA_Report_2024.pdf,35,0.6607,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,0.6226,"Year Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 32,016 $ 5,083 $ 26,933 530 %\nGraphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."
3,NVIDIA_Report_2024.pdf,35,0.5638,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nRevenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including aut..."


## Complete Evaluation 

In [36]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Basic_RAG_Semantic",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Basic_RAG_Semantic-ca34b9a9' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=e0e69b28-0c4d-4030-bdb4-39969cbc98c0




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,1.0026,"Our business continuity and disaster recovery planning may not be sufficient for all eventualities. Geopolitical anddomestic political developments and other events beyond our control, can increase economic volatility globally. Political instability, changes in government or\nadverse political developments in or around any of the major countries in which we do business may harm our business, financial condition and results ofoperations. Worldwide geopolitical tensions and conflicts, including but not limited to China, Hong Kong, Israel, Korea and Taiwan where the manufacture of ourproduct components and final assembly of our products are concentrated may result in changing regulatory requirements, and other disruptions that could\nimpact our operations and operating strategies, product demand, access to global markets, hiring, and profitability. For example, other countries have restrictedand may continue in the future to restrict business with the State of Israel, where we have engineer..."
1,Unknown Document,N/A,1.0000,"Table of Contents\nWe are monitoring the impact of the geopolitical conflict in and around Israel on our operations, including the health and safety of our approximately 3,700employees in the region who primarily support the research and development, operations, and sales and marketing of our networking products. Our operatingexpenses in fiscal year 2024 include expenses for financial support to impacted employees and charitable activity. We believe our global supply chain for our\nnetworking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or others\nmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense toour business; however, if the conflict is fu..."
2,Unknown Document,N/A,0.9393,"As of the end of fiscal year2024, we had approximately 29,600 employees in 36 countries, 22,200 were engaged in research and development and 7,400 were engaged in sales,\nmarketing, operations, and administrative positions. The Compensation Committee of our Board of Directors assists in the oversight of policies and strategiesrelating to human capital management. To be competitive and execute our business strategy successfully, we must recruit, develop, and retain talented employees, including qualified executives,\nscientists, engineers, and technical and non-technical staff. Recruitment\nAs the demand for global technical talent continues to be competitive, we have grown our technical workforce and have been successful in attracting top talentto NVIDIA. We have attracted talent globally through our strong employer brand and differentiated hiring strategies for college, professional, and leadershiptalent. Our workforce is 83% technical and 49% hold advanced degrees. Additionally, we have..."
3,Unknown Document,N/A,0.7186,"Within our supply chain, we continuously\nmanage product availability and costs with our vendors. Israel and Hamas Conflict\nWe are monitoring the impact of the geopolitical conflict in and around Israel on our operations, including the health and safety of our approximately 3,700\nemployees in the region who primarily support the research and development, operations, and sales and marketing of our networking products. Our operatingexpenses in fiscal year 2024 include expenses for financial support to impacted employees and charitable activity. We believe our global supply chain for our\nnetworking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military d

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.8634,"Gaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators. Our gaming platforms leverage our GPUs and sophisticated software to enhance the gaming experience with smoother, higher quality graphics. We developed\nNVIDIA RTX to bring next generation graphics and AI to games. NVIDIA RTX features ray tracing technology for real-time, cinematic-quality rendering. Raytracing, which has long been used for special effects in the movie industry, is a computationally intensive technique that simulates the physical behavior of lightto achieve greater realism in computer-generated scenes. NVIDIA RTX also features deep learning super sampling, or NVIDIA DLSS, our AI technology that\nboosts frame rates w..."
1,Unknown Document,N/A,0.7847,"Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications. Our two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information. Headquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998. Recent Developments, Future Objectives and Challenges\nDemand and Supply, Product Transitions, and New Products and Business Models\nDemand for our data center systems and products surged in fiscal year 2024...."
2,Unknown Document,N/A,0.7467,"In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises. Headquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998. Our Businesses\nWe report our business results in two segments. The Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services. The Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software fo..."
3,Unknown Document,N/A,0.7014,"Revenue by geographic areas was as follows: \n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nUnited States $ 26,966 $ 8,292 $ 4,349 \nTaiwan 13,405 6,986 8,544 \nChina (including Hong Kong) 10,306 5,785 7,111 \nOther countries 10,245 5,911 6,910 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \nRevenue from sales to customers outside of the United States accounted for 56%, 69%, and 84% of total revenue for fiscal years 2024, 2023, and 2022,\nrespectively. The increase in revenue to the United States for fiscal year 2024 was primarily due to higher U.S.-based Compute & Networking segment demand. Sales to one customer represented 13% of total revenue for fiscal year 2024, which was attributable to the Compute & Networking segment. No customerrepresented 10% or more of total revenue for fiscal years 2023 and 2022. The following table summarizes information pertai

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.7540,"Table of Contents\nThe following table sets forth, for the periods indicated, certain items in our Consolidated Statements of Income expressed as a percentage of revenue. Year Ended\n Jan 28, 2024 Jan 29, 2023\nRevenue 100.0 % 100.0 %\nCost of revenue 27.3 43.1 \nGross profit 72.7 56.9 \nOperating expenses \nResearch and development 14.2 27.2 \nSales, general and administrative 4.4 9.1 \nAcquisition termination cost — 5.0 \nTotal operating expenses 18.6 41.3 \nOperating income 54.1 15.6 \nInterest income 1.4 1.0 \nInterest expense (0.4) (1.0)\nOther, net 0.4 (0.1)\nOther income (expense), net 1.4 (0.1)\nIncome before income tax 55.5 15.5 \nIncome tax expense (benefit) 6.6 (0.7)\nNet income 48.9 % 16.2 %\nReportable Segments\nRevenue by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 47,405 $ 15,068 $ 32,337 215 %\nGraphics 13,517 11,906 1,611 14 %\nTotal $ 60,922 $ 26,974 $ 33,948 126 %\nOperating Income by Reportable Segments\nYear Ended\nJan 28..."
1,Unknown Document,N/A,0.7255,"The increase primarily reflected growth in self-driving platforms. Gross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory provisions as a percentage of revenue. Operating expenses increased for fiscal year 2024, driven by growth in employees and compensation increases. Fiscal year 2023 also included a $1.4 billion\nacquisition termination charge related to the proposed Arm transaction. Market Platform Highlights\nData Center revenue for fiscal year 2024 was $47.5 billion, up 217% from fiscal year 2023. In Data Center, we launched AI inference platforms that combine our\nfull-stack inference software with NVIDIA Ada, NVIDIA Hopper and NVIDIA Grace Hopper processors optimized for generative AI, LLMs and other AI workloads.We introduced NVIDIA DGX Cloud and AI Foundations to help businesses create and operate custom large language models and generative AI models. As AV\nalgorithms move to video transformers, and more cars are equippe..."
2,Unknown Document,N/A,0.7159,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive. Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago. Data Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including aut..."
3,Unknown Document,N/A,0.6228,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Balance Sheets(In millions, except par value)\n Jan 28, 2024 Jan 29, 2023\nAssets \nCurrent assets: \nCash and cash equivalents $ 7,280 $ 3,389 \nMarketable securities 18,704 9,907 \nAccounts receivable, net 9,999 3,827 \nInventories 5,282 5,159 \nPrepaid expenses and other current assets 3,080 791 \nTotal current assets 44,345 23,073 \nProperty and equipment, net 3,914 3,807 \nOperating lease assets 1,346 1,038 \nGoodwill 4,430 4,372 \nIntangible assets, net 1,112 1,676 \nDeferred income tax assets 6,081 3,396 \nOther assets 4,500 3,820 \nTotal assets $ 65,728 $ 41,182 \nLiabilities and Shareholders' Equity\nCurrent liabilities: \nAccounts payable $ 2,699 $ 1,193 \nAccrued and oth

...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.9619,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated. Signature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024\nJohn O. Dabiri \n/s/ PERSIS DRELL Director February 21, 2024\nPersis Drell\n/s/ DAWN HUDSON Director February 21, 2024\nDawn Hudson\n/s/ HARVEY C. JONES Director February 21, 2024\nHarvey C...."
1,Unknown Document,N/A,0.9599,"Signatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024. NVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney\nKNOW ALL PERSONS BY THESE PRESENTS, that each person whose signature appears below constitutes and appoints Jen-Hsun Huang and Colette M.Kress, and each or any one of them, his true and lawful attorney-in-fact and agent, with full power of substitution and resubstitution, for him and in his name,\nplace and stead, in any and all capacities, to sign any and all amendments to this report, and to file the same, with all exhibits thereto, and other documents inconnection therewith, with the Securities and Exchange Commission, granting unto said attorneys-in-facts and agents, and each of them, full power andauthority to do and perform each a..."
2,Unknown Document,N/A,0.9101,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University. Ajay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr. Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinne..."
3,Unknown Document,N/A,0.7295,"Table of Contents\nOur support is enhanced during times of crisis, such as war or economic volatility, to take care of our existing team of world-class talent and their families. Diversity, Inclusion, and Belonging\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees. When recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in thetechnology field, including women, Black/African American, and Hispanic/Latino candidates. To this end, we have been:\n• Partnering with institutions and professional organizations serving historically underrepresented communities;\n• Embedding dedicated recruiting teams to business areas to shepherd u

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.7347,"Table of Contents\nGovernment Regulations\nOur worldwide business activities are subject to various laws, rules, and regulations of the United States as well as of foreign governments. During the third quarter of fiscal year 2023, the U.S. government, or the USG, announced licensing requirements that, with certain exceptions, impact exports to\nChina (including Hong Kong and Macau) and Russia of our A100 and H100 integrated circuits, DGX or any other systems or boards which incorporate A100 orH100 integrated circuits. In July 2023, the USG informed us of an additional licensing requirement for a subset of A100 and H100 products destined to certain customers and otherregions, including some countries in the Middle East. In October 2023, the USG announced new and updated licensing requirements that became effective in our fourth quarter of fiscal year 2024 for exports to\nChina and Country Groups D1, D4, and D5 (including but not limited to Saudi Arabia, the United Arab Emirates, and Vietnam..."
1,Unknown Document,N/A,0.6669,"Table of Contents\nGlobal Trade\nDuring the third quarter of fiscal year 2023, the USG, announced licensing requirements that, with certain exceptions, impact exports to China (including Hong\nKong and Macau) and Russia of our A100 and H100 integrated circuits, DGX or any other systems or boards which incorporate A100 or H100 integrated circuits. In July 2023, the USG informed us of an additional licensing requirement for a subset of A100 and H100 products destined to certain customers and other\nregions, including some countries in the Middle East. In October 2023, the USG announced new and updated licensing requirements that became effective in our fourth quarter of fiscal year 2024 for exports toChina and Country Groups D1, D4, and D5 (including but not limited to Saudi Arabia, the United Arab Emirates, and Vietnam, but excluding Israel) of our\nproducts exceeding certain performance thresholds, including A100, A800, H100, H800, L4, L40, L40S and RTX 4090. The licensing requirements also..."
2,Unknown Document,N/A,0.6498,"Table of Contents\ndisadvantage NVIDIA against certain of our competitors who sell products that are not subject to the new restrictions or may be able to acquire licenses for theirproducts. Management of these new licenses and other requirements is complicated and time consuming. Our competitive position has been harmed, and our competitiveposition and future results may be further harmed, over the long-term, if there are further changes in the USG’s export controls, including further expansion of thegeographic, customer, or product scope of the controls, if customers purchase product from competitors, if customers develop their own internal solution, if we\nare unable to provide contractual warranty or other extended service obligations, if the USG does not grant licenses in a timely manner or denies licenses tosignificant customers or if we incur significant transition costs. Even if the USG grants any requested licenses, the licenses may be temporary or impose\nburdensome conditions t..."
3,Unknown Document,N/A,0.5570,"Table of Contents\nproducts, and it is likely that additional unilateral or multilateral controls will be adopted. Such controls have been and may again be very broad in scope and\napplication, prohibit us from exporting our products to any or all customers in one or more markets, including but not limited to China, and could negativelyimpact our manufacturing, testing and warehousing locations and options, or could impose other conditions that limit our ability to serve demand abroad andcould negatively and materially impact our business, revenue and financial results. Export controls targeting GPUs and semiconductors associated with AI,\nwhich have been imposed and are increasingly likely to be further tightened, would further restrict our ability to export our technology, pro

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.9301,"Colette M. Kress joined NVIDIA in 2013 as Executive Vice President and Chief Financial Officer. Prior to NVIDIA, Ms. Kress most recently served as Senior\nVice President and Chief Financial Officer of the Business Technology and Operations Finance organization at Cisco Systems, Inc., a networking equipmentcompany, since 2010. At Cisco, Ms. Kress was responsible for financial strategy, planning, reporting and business development for all business segments,\nengineering and operations. From 1997 to 2010 Ms. Kress held a variety of positions at Microsoft, a software company, including, beginning in 2006, ChiefFinancial Officer of the Server and Tools division, where Ms. Kress was responsible for financial\n12..."
1,Unknown Document,N/A,0.8715,"Timothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General Counsel andSecretary in February 2018. Prior to NVIDIA, Mr. Teter spent more than two decades at the law firm of Cooley LLP, where he focused on litigating patent and\ntechnology related matters. Prior to attending law school, he worked as an engineer at Lockheed Missiles and Space Company, an aerospace company. Mr.Teter holds a B.S. degree in Mechanical Engineering from the University of California at Davis and a J.D...."
2,Unknown Document,N/A,0.8206,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University. Ajay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr. Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinne..."
3,Unknown Document,N/A,0.7465,"Table of Contents\nOur support is enhanced during times of crisis, such as war or economic volatility, to take care of our existing team of world-class talent and their families. Diversity, Inclusion, and Belonging\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees. When recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in thetechnology field, including women, Black/African American, and Hispanic/Latino candidates. To this end, we have been:\n• Partnering with institutions and professional organizations serving historically underrepresented communities;\n• Embedding dedicated recruiting teams to business areas to shepherd underrepresented candidates through the interview process and find internalopportunities;\n• Supporting the development of women employees through programs aimed at building a pipeline of future leader..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.9449,"degree from Harvard Business School. Debora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement\nsolutions and optical products for the telecommunications industry. She served from 2002 to 2004 as Senior Vice President and General Manager of the Electro-Optics business at Coherent, Inc., a manufacturer of commercial and scientific laser equipment. Previously, she worked at Quantum Corp., a data protectioncompany, as President of the Personal Computer Hard Disk Drive Division, and at Hewlett-Packard. Ms. Shoquist holds a B.S. degree in Electrical Engineering\nfrom Kansas State University and a B.S. degree in Biology from Santa Clara University...."
1,Unknown Document,N/A,0.8785,"Timothy S. Teter joined NVIDIA in 2017 as Senior Vice President, General Counsel and Secretary and became Executive Vice President, General Counsel andSecretary in February 2018. Prior to NVIDIA, Mr. Teter spent more than two decades at the law firm of Cooley LLP, where he focused on litigating patent and\ntechnology related matters. Prior to attending law school, he worked as an engineer at Lockheed Missiles and Space Company, an aerospace company. Mr.Teter holds a B.S. degree in Mechanical Engineering from the University of California at Davis and a J.D...."
2,Unknown Document,N/A,0.8317,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University. Ajay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr. Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinne..."
3,Unknown Document,N/A,0.6857,"Table of Contents\nOur support is enhanced during times of crisis, such as war or economic volatility, to take care of our existing team of world-class talent and their families. Diversity, Inclusion, and Belonging\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees. When recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in thetechnology field, including women, Black/African American, and Hispanic/Latino candidates. To this end, we have been:\n• Partnering with institutions and professional organizations serving historically underrepresented communities;\n• Embedding dedicated recruiting teams to business areas to shepherd underrepresented candidates through the interview process and find internalopportunities;\n• Supporting the development of women employees through programs aimed at building a pipeline of future leader..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.6580,"Table of Contents\nResearchers and developers use our computing solutions to accelerate a wide range of important applications, from simulating molecular dynamics to climate\nforecasting. With support for more than 3,500 applications, NVIDIA computing enables some of the most promising areas of discovery, from climate prediction tomaterials science and from wind tunnel simulation to genomics. Including GPUs and networking, NVIDIA powers over 75% of the supercomputers on the global\nTOP500 list, including 24 of the top 30 systems on the Green500 list. Gamers choose NVIDIA GPUs to enjoy immersive, increasingly cinematic virtual worlds. In addition to serving the growing number of gamers, the market for PCGPUs is expanding because of the burgeoning population of live streamers, broadcasters, artists, and creators. With the advent of generative AI, we expect a\nbroader set of PC users to choose NVIDIA GPUs for running generative AI applications locally on their PC, which is critical for privac..."
1,Unknown Document,N/A,0.6388,"Indeed, it can help solve problems that were previously deemed unsolvable. We work todeliver continued performance leaps that outpace Moore’s Law by leveraging innovation across the architecture, chip design, system, interconnect, and software\nlayers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partne..."
2,Unknown Document,N/A,0.6113,"In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises. Headquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998. Our Businesses\nWe report our business results in two segments. The Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services. The Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software fo..."
3,Unknown Document,N/A,0.5415,"Gaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators. Our gaming platforms leverage our GPUs and sophisticated software to enhance the gaming experience with smoother, higher quality graphics. We developed\nNVIDIA RTX to bring next generation graphics and AI to games. NVIDIA RTX features ray tracing technology for real-time, cinematic-quality rendering. Raytracing, which has long been used for special effects in the movie industry, is a computationally intensive technique that s